In this assignment I will explore and cluster the neigbourhoods in Toronto. In this first assignment of this whole assignment the goal is to get the data from  Wikipedia page, transform it to a pandas dataframe and clean it so that the data is in a structured format. Lets begin.

In [244]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [58]:
# at first i import needed libraries 
import pandas as pd
import numpy as np

# !conda install -c conda-forge folium=0.5.0 --yes
# import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [245]:
# then I wrangle the data from Wikibedia page
data_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto_data = pd.read_html(data_path)


After I wrangled the data I checked in what format the data is (result is a list) and how many datasets does the data include (result was that the lenght of the dataset is 3). I also checked is the list on what format, and the result of this was a pandas Dataframe. 

So no I need to transform the first list to a dataframe called df_toronto_data. I also checked that does the data include duplicates in Postal Codes or Borough or Neighborhood - and the result is no. I will also clean the celss with borought such as Not assigned.

In [246]:
#trnasforming the data form first list into a Dataframe
df_toronto_data = toronto_data[0]

In [247]:
df_toronto_data = df_toronto_data.drop(df_toronto_data.loc[df_toronto_data["Borough"]=="Not assigned"].index, inplace=False).reset_index(drop = True)

print(df_toronto_data.shape)

df_toronto_data.head(12)



(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Created Dataframe consist three colums: PostalCode, Borough and Neighborhood
I have now ignored celss with a borought that is Not assingned.
In the data there is no twice listed PostalCodes - Neighborhoods which have the same PostalCode are mentioned in the same PostalCode (rows are combined into one row, separated with a comma)
Cells do not have Not assingned text either in Borough or in Neigborhood.


In this second part of the assignment the mission is to built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. Because the given package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, I will use this link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data.

So now I need to read the csv file and then merge the DataFrame whit df_toronto_data. The result will be a new DataFrame including information and data drom both datasets.

In [248]:
# at first I read the csv file
data_path = "http://cocl.us/Geospatial_data"
geogordinates_toronto = pd.read_csv(data_path)

After I readed the file I checked the len of the file (103), type of the file (pandas.core.frame.DataFrame) and shape of the file ((103, 3). Data is same type as the previos and the shape of the file is the same than in df_toroto_data so I can go on and unite these to datasets.

In [249]:
# uniting these two datasets
df_data_toronto = df_toronto_data.merge(geogordinates_toronto, left_on="Postal Code", right_on="Postal Code")

print(df_data_toronto.shape)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_data_toronto['Borough'].unique()),
        df_data_toronto.shape[0]
    )
)

df_data_toronto.head(12)

(103, 5)
The dataframe has 10 boroughs and 103 neighborhoods.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In this third and last part of the whole assingment, my mission is to explore and cluster the neighborhoods in Toronto. In this section I choose to works only with boroughs that contain the word Toronto. I will start by visualizising to map every boroughs and then I will clean the data to include only boroughs that contain the word Toronto and visualize those boroughs by map.

In [250]:
# at first I have to get the geograpical coordinate of Toronto and for that I will use Nominatim from geopy.geocoders

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [251]:
# creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_data_toronto['Latitude'], df_data_toronto['Longitude'], df_data_toronto['Borough'], df_data_toronto['Neighborhood']):
    label = '{}, {}'.format(df_data_toronto, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [252]:
# picking up only the data from Boroughs wich contain word Toronto -"Central Toronto", "Downtown Toronto", "East Toronto", "West Toronto"

list_borough = ["Central Toronto", "Downtown Toronto", "East Toronto", "West Toronto"]

toronto_data = df_data_toronto[df_data_toronto["Borough"].isin(list_borough)].reset_index(drop=True)
toronto_data.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [253]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)


The dataframe has 4 boroughs and 39 neighborhoods.


So, no I have dataset wich contains the information of boroughs (which containts the word of Toronto), neighborhoods, latitudes and longitudes of Toronto and I can go on, and start to explore and cluster the data.

To do that I will need to use Foursquare data. At first I will explore only one neighborhood and then all of selected boroughs together. So lets start.

In [255]:
# at first I have to give my credentials for Foursquare using
import getpass
CLIENT_ID = getpass.getpass('client_id: ')

client_id: ········


In [256]:
CLIENT_SECRET = getpass.getpass('client_secret: ')

client_secret: ········


In [257]:
# then I will choose only one Neighborhood from the dataset

toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [258]:
# now I get the neighborhoods latitude and longitude out from the dataset

neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [259]:
# then I will get the data of the top 100 Venues (of the places in 500 metres from Park, Harbourfront) from Foursquare 

LIMIT = 100

radius = 1000

latitude = 43.7280205
longitude = -79.3887901

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?client_id=Y43CZE3Y3GG0ZGX0JSGEAOZN252FLXYGI5UQM4URMA25SO2J&client_secret=JFSW4ER3HVC51D3HFIRYDZXCRVLDBRAORR4PLSCCGDQJSGTQ&ll=43.7280205,-79.3887901&v=20180605&radius=1000&limit=100'

In [260]:
# now I will send the GET request !!!! MIKSI TUOTA JSON JUTTUA KÄYTETÄÄN TÄSSÄ?

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f06ff3a8a993d29725310b2'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.73702050900001,
    'lng': -79.37635882105461},
   'sw': {'lat': 43.71902049099999, 'lng': -79.40122137894538}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 4

In [261]:
# Information is in the item key. Before I can proceed, I have to define the function to do that. Function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [262]:
# no I can clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Granite Club,Gym / Fitness Center,43.733043,-79.381986
2,Tim Hortons,Coffee Shop,43.727324,-79.379563
3,Granite Club President's Lounge,Café,43.733005,-79.382059
4,Glendon Bookstore,Bookstore,43.727024,-79.378976
5,Glendon Forest,Trail,43.727226,-79.378413
6,Glendon Rose Garden,College Quad,43.727334,-79.378222
7,Lunik Co-op,Café,43.727311,-79.377835
8,Glendon Athletic Club,College Gym,43.728799,-79.376819


In [263]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


It seems that in the neighborhood Regent Park, Harbourfront, there is only 4 venues in 500 m radius from the centre. I tested are there much more, if I change the radius in to 1 000 m. Result is shown here and the answer is then 9 venues. Venues are school, bookstore, park cafe and trail. From this data I make my assumption that there is really not much venues to visit. 

So lets strat to explore and cluster together all boroughs (which containts the word of Toronto).

To do that at first I have to create a function to reoeat the same process to alla the neighborhoods in selected dataset.

In [267]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
                # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [268]:
# then I will look the names of the neigborhoods

toronto_central_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [269]:
print(toronto_central_venues.shape)
toronto_central_venues.head()

(1624, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


At this point I now (I did the checking earlier) that there is 4 boroughs and 39 neighborhoods in toronto_data set. From this dataset I found 1624 venues in 500 m radius of the neighbourhoods centers and It now contains (in dataframe toronto_central_venues) geograpical coordinates of neighbourhood where venue is and of venues it selfs. It also includes the categories the venues.

After exploring the data, I found out that there where one venue category too much, somehow Neighborhood was listet also to venue category. That is why i cleaned the data of them.

In [270]:
# venue Category Neighborhood ruins the analysis later on --> Delete the rows from venues dataframe
toronto_central_venues[toronto_central_venues['Venue Category']=='Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
262,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
414,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
516,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
952,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


In [271]:
#shape on orginal dataset
toronto_central_venues.shape

(1624, 7)

In [272]:
#cleanin out of venues the category Neighbourhood and gettin cleaner dataset to continue whit

toronto_central_venues = toronto_central_venues[toronto_central_venues['Venue Category']!='Neighborhood']
toronto_central_venues.shape

(1620, 7)

In [273]:
# now I will goupby and count venues in each neigborhoods

toronto_central_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,69,69,69,69,69,69
Christie,17,17,17,17,17,17
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [274]:
# now i calculate how many unique categories does the dataframe include

print('There are {} uniques categories.'.format(len(toronto_central_venues['Venue Category'].unique())))

There are 224 uniques categories.


In [275]:
for col in toronto_central_venues['Venue Category'].unique():
    if col == 'Neighborhood':
        print(col)

At this point I can say, that there are 1624 venues and 224 different/unique types of venue categories in dataset. Biggest amount of venues are in neighborhoods Toronto Dominion Centre, Design Exchange, Stn A PO Boxes, Richmond, Adelaide, King, Harbourfront East, Union Station, Toronto Islands, Garden District, Ryerson, Commerce Court,Victoria Hotel. Smallest amount of venues are in neighborhoods Forest Hill North & West, Forest Hill Road Park, The Beaches, Roselawn, Rosedale, Moore Park, Summerhill East, Lawrence Park. Based on this i would prefer to visit the first neighborhoods if I go to Toronto and want to do something. Top of venue categories are in Harbourfront East, Union Station, Toronto Islands, Garden District, Ryerson and Commerce Court, Victoria Hotel, so if I want to do lots of different kinds of things it would probably be the best to visit those neighborhoods.

But before making more assumptions, lets explore neigborhoods more.


In [276]:

# now I will make a new dataframe wich gives the value 1 or 0 to evevry venye = tels which category venue belongs to
# one hot encoding
toronto_central_onehot = pd.get_dummies(toronto_central_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_central_onehot['Neighborhood'] = toronto_central_venues['Neighborhood']
fixed_columns = [toronto_central_onehot.columns[-1]] + list(toronto_central_onehot.columns[:-1])
toronto_central_onehot = toronto_central_onehot[fixed_columns]
toronto_central_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [277]:
toronto_central_onehot.shape

(1620, 225)

No i have dataset telling me in wich category of venues venue belogs to and I can calculate the frequency of venue categories in each neighborhoods

In [278]:
# grouping neighborhoods and taking the mean of frequency of occurence of each categories

toronto_central_grouped = toronto_central_onehot.groupby('Neighborhood').mean().reset_index()
toronto_central_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [279]:
toronto_central_grouped.shape

(39, 225)

At this point of this assingment I can go on exploring the most most common venues by printing out the most common venues in each neighborhood. On the result on this (below) I can say, that there is some neighborhoods wich come out from data clearly. For example CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport neigborhood is in practise area almoust only for Airport and its services.

Cause I want to see differences between these Neighbourhoods I will know make a DataFrame showing 10 top venues in each neighborhoods.

In [280]:
# printing neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_central_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_central_grouped[toronto_central_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2      Restaurant  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1         Breakfast Spot  0.08
2  Performing Arts Venue  0.08
3            Coffee Shop  0.08
4          Grocery Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
            venue  freq
0             Spa  0.07
1      Restaurant  0.07
2  Farmers Market  0.07
3   Burrito Place  0.07
4      Skate Park  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0

In [281]:
# writing a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [282]:
# creating the new dataframe and displaying the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_central_venues_sorted = pd.DataFrame(columns=columns)
toronto_central_venues_sorted['Neighborhood'] = toronto_central_grouped['Neighborhood']

for ind in np.arange(toronto_central_grouped.shape[0]):
    toronto_central_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_central_grouped.iloc[ind, :], num_top_venues)

toronto_central_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Pharmacy,Beer Bar,Bakery,Seafood Restaurant,Café,Cheese Shop,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Breakfast Spot,Coffee Shop,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Farmers Market,Park,Brewery,Light Rail Station,Comic Shop,Gym / Fitness Center,Garden Center,Garden,Pizza Place,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Thai Restaurant,Salad Place


Abow is DataFrame showing neighbourhoods and their most common venues on descending order based on the mean of frequency of occurence of each categories.

Because I know want to make conclusion about neigborhoods venues, I will start clustering these result by kmeans. I clustered labels and ended up to use 6 labels. Data in Toronto Centre (boroughs with name Toronto) are in the centre of Toronto and that is why they are really simular to each other. Clusterig by more labels does not change the result much and with les than 6 labels result is only parks separeted from others.

With 6 labels result are most of label in label 0 (32 neigbourhoods), second highest is label 4 (3 neigbourhoods) and other labels include only 1 neigbourhood. 

After that I join clustered neighborhoods dataset to most common venues by neighborhoods data set and replace in that DataFrame numbers of labels by names.

In [283]:
# set number of clusters
kclusters = 6

toronto_central_grouped_clustering = toronto_central_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_central_grouped_clustering)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_

for label in range(kclusters):
    cnt = list(kmeans.labels_).count(label)
    print(f"Label {label} count: {cnt}")

Label 0 count: 32
Label 1 count: 1
Label 2 count: 1
Label 3 count: 1
Label 4 count: 3
Label 5 count: 1


In [284]:
toronto_central_merged.shape

(39, 16)

In [285]:
# add clustering labels
toronto_central_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_central_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_central_merged = toronto_central_merged.join(toronto_central_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_central_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Bookstore,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Creperie,Moroccan Restaurant,Lingerie Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Trail,Pub,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [286]:
# labels are shown in numbers. For chart is more practically to use names, so I named the different types of labels

cluster_dict = { 0:'shopping area', 1:'free time zone' , 2:"Swim School", 3:"living neighbourhood", 4:"parks and playgrounds" , 5:"Airport"} 
toronto_central_merged["Cluster Labels"] = toronto_central_merged["Cluster Labels"].map(cluster_dict)


In [287]:
# dataset with label names
toronto_central_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,shopping area,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,shopping area,Coffee Shop,Diner,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,shopping area,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Bookstore,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,shopping area,Coffee Shop,Café,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Creperie,Moroccan Restaurant,Lingerie Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,living neighbourhood,Health Food Store,Trail,Pub,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [288]:
clusterback_dict = {'shopping area':0, 'free time zone':1 , "Swim School":2, "living neighbourhood":3, "parks and playgrounds":4 , "Airport":5} 
toronto_central_merged["Cluster Labels"] = toronto_central_merged["Cluster Labels"].map(clusterback_dict)


Now I have table with clustered information of venues in boroughts in Toronto which include the word Toronto

Lets create a map of clustered neighbourhoods.

In [289]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_central_merged['Latitude'], toronto_central_merged['Longitude'], toronto_central_merged['Neighborhood'], toronto_central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In the map shown abouw you can see the result of this assingnment.

Red markes (label 0) are neighbourhoods were you can shop and dine or visit perheps a caffee. If you want to visit a park I regommentate to visit the neighbourhoods with green marker (label 4). Airpot is in neighbourhood marked orange (label 5) and there is actually nothing else, or atleast venues.

Label 2 (blue), label 3 (light green) and label 1 (violet) are neighbourhoods with not so many shops than red ones - but city area still. In label 2 you will find a Swin School and Train, from label 1 garden and dancing studios and coffee shops and from label 3 is basicly neihgborhood for just living with some coffee shops for example.

If you want to know more of label - read untill the end - abouw there is information of each neighbourhoods 10 most common venues sortet by labels.

In [290]:
# information of label 0 =red neighbourhoods
toronto_central_merged.loc[toronto_central_merged['Cluster Labels'] == 0, toronto_central_merged.columns[[1] + list(range(5, toronto_central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,Downtown Toronto,0,Coffee Shop,Diner,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Bookstore,Diner
3,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Creperie,Moroccan Restaurant,Lingerie Store
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Restaurant,Pharmacy,Beer Bar,Bakery,Seafood Restaurant,Café,Cheese Shop,Farmers Market
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Thai Restaurant,Salad Place
7,Downtown Toronto,0,Grocery Store,Café,Park,Athletics & Sports,Diner,Italian Restaurant,Candy Store,Baby Store,Restaurant,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Thai Restaurant,Clothing Store,Gym,Deli / Bodega,Sushi Restaurant,Pizza Place
9,West Toronto,0,Bakery,Pharmacy,Middle Eastern Restaurant,Bank,Bar,Café,Furniture / Home Store,Supermarket,Music Venue,Grocery Store
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Fried Chicken Joint,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery


In [291]:
# information of label 1 = violet neighbourhoods
toronto_central_merged.loc[toronto_central_merged['Cluster Labels'] == 1, toronto_central_merged.columns[[1] + list(range(5, toronto_central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Garden,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [292]:
# information of label 2 blue neighbourhoods
toronto_central_merged.loc[toronto_central_merged['Cluster Labels'] == 2, toronto_central_merged.columns[[1] + list(range(5, toronto_central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Park,Swim School,Bus Line,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [293]:
# information of label 3 =light green neighbourhoods
toronto_central_merged.loc[toronto_central_merged['Cluster Labels'] == 3, toronto_central_merged.columns[[1] + list(range(5, toronto_central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Health Food Store,Trail,Pub,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [294]:
# information of label 4 = green neighbourhoods
toronto_central_merged.loc[toronto_central_merged['Cluster Labels'] == 4, toronto_central_merged.columns[[1] + list(range(5, toronto_central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,4,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
29,Central Toronto,4,Park,Playground,Trail,Tennis Court,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
33,Downtown Toronto,4,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [295]:
# information of label 5 =orange neighbourhoods
toronto_central_merged.loc[toronto_central_merged['Cluster Labels'] == 5, toronto_central_merged.columns[[1] + list(range(5, toronto_central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,5,Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
